<a href="https://colab.research.google.com/github/gaiamilenium99/ai4eng_health_issues1/blob/main/Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Procesamiento del dataset



1.   Importar dataset

2.   Observar valores null de la informacion


In [5]:
import numpy as np
import pandas as pd
import io
from google.colab import files

In [1]:
! pip install kaggle
! mkdir ~/.kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download mitishaagarwal/patient

 60% 5.00M/8.30M [00:00<00:00, 50.6MB/s]
100% 8.30M/8.30M [00:00<00:00, 74.1MB/s]


In [4]:
! unzip patient.zip

Archive:  patient.zip
replace dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset.csv             


# 1. Lectura del Dataset

In [6]:
df=pd.read_csv("/content/dataset.csv")

In [9]:
df.shape

(91713, 85)

In [11]:
print(df)

       encounter_id  patient_id  hospital_id   age        bmi  \
0             66154       25312          118  68.0  22.730000   
1            114252       59342           81  77.0  27.420000   
2            119783       50777          118  25.0  31.950000   
3             79267       46918          118  81.0  22.640000   
4             92056       34377           33  19.0        NaN   
...             ...         ...          ...   ...        ...   
91708         91592       78108           30  75.0  23.060250   
91709         66119       13486          121  56.0  47.179671   
91710          8981       58179          195  48.0  27.236914   
91711         33776      120598           66   NaN  23.297481   
91712          1671       53612          104  82.0  22.031250   

       elective_surgery  ethnicity gender  height           icu_admit_source  \
0                     0  Caucasian      M   180.3                      Floor   
1                     0  Caucasian      F   160.0          

Se observa la cantidad de valores nulos presentes en el dataset

In [8]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Data columns (total 85 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   encounter_id                   91713 non-null  int64  
 1   patient_id                     91713 non-null  int64  
 2   hospital_id                    91713 non-null  int64  
 3   age                            87485 non-null  float64
 4   bmi                            88284 non-null  float64
 5   elective_surgery               91713 non-null  int64  
 6   ethnicity                      90318 non-null  object 
 7   gender                         91688 non-null  object 
 8   height                         90379 non-null  float64
 9   icu_admit_source               91601 non-null  object 
 10  icu_id                         91713 non-null  int64  
 11  icu_stay_type                  91713 non-null  object 
 12  icu_type                       91713 non-null 

<ipython-input-8-f6a666cc1a3a>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose = True, null_counts = True)


Eliminamos la columna 83 ya que no se encuentra caracterizada

In [12]:
df = df.drop(['Unnamed: 83'], axis = 1)

Eliminamos los datos que no consideramos importantes para el modelo que se desea plantear y analizar

In [13]:
df.drop(['encounter_id','patient_id', 'icu_admit_source','icu_id', 'icu_stay_type', 'icu_type'], axis = 1, inplace = True)

In [14]:
df.drop(['hospital_id','apache_2_diagnosis','apache_3j_diagnosis','apache_4a_hospital_death_prob','apache_4a_icu_death_prob','apache_2_bodysystem','apache_3j_bodysystem'], axis = 1, inplace = True)

In [15]:
df.nunique()

hospital_id                      147
age                               74
bmi                            34888
elective_surgery                   2
ethnicity                          6
                               ...  
immunosuppression                  2
leukemia                           2
lymphoma                           2
solid_tumor_with_metastasis        2
hospital_death                     2
Length: 72, dtype: int64

Al observar el dataset es importante notar que los registros no presentan razgos que permitan agruparlos adecuadamente para completar informacion faltante en campos como peso, edad, o estatura. Por esto mismo, es necesario eliminar estos registros para no afectar la construccion del modelo

In [20]:
df = df[df[['bmi','age', 'weight', 'height']].isna().sum(axis=1) == 0]
df

,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,pre_icu_los_days,weight,apache_post_operative,...,d1_potassium_min,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,118,68.0,22.730000,0,Caucasian,M,180.3,0.541667,73.9,0,...,3.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,81,77.0,27.420000,0,Caucasian,F,160.0,0.927778,70.2,0,...,3.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,118,25.0,31.950000,0,Caucasian,F,172.7,0.000694,95.3,0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,118,81.0,22.640000,1,Caucasian,F,165.1,0.000694,61.7,1,...,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,83,67.0,27.560000,0,Caucasian,M,190.5,0.000694,100.0,0,...,3.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91706,121,54.0,19.770448,0,Native American,M,177.8,0.025694,62.5,0,...,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91708,30,75.0,23.060250,0,Caucasian,M,177.8,0.298611,72.9,0,...,4.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
91709,121,56.0,47.179671,0,Caucasian,F,183.0,0.120139,158.0,0,...,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91710,195,48.0,27.236914,0,Caucasian,M,170.2,0.046528,78.9,0,...,3.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [21]:
df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84229 entries, 0 to 91712
Data columns (total 72 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   hospital_id                  84229 non-null  int64  
 1   age                          84229 non-null  float64
 2   bmi                          84229 non-null  float64
 3   elective_surgery             84229 non-null  int64  
 4   ethnicity                    83060 non-null  object 
 5   gender                       84219 non-null  object 
 6   height                       84229 non-null  float64
 7   pre_icu_los_days             84229 non-null  float64
 8   weight                       84229 non-null  float64
 9   apache_post_operative        84229 non-null  int64  
 10  arf_apache                   84229 non-null  float64
 11  gcs_eyes_apache              83149 non-null  float64
 12  gcs_motor_apache             83149 non-null  float64
 13  gcs_unable_apach

<ipython-input-21-f6a666cc1a3a>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose = True, null_counts = True)


In [22]:
df.dropna()

,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,pre_icu_los_days,weight,apache_post_operative,...,d1_potassium_min,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,118,68.0,22.730000,0,Caucasian,M,180.3,0.541667,73.9,0,...,3.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,81,77.0,27.420000,0,Caucasian,F,160.0,0.927778,70.2,0,...,3.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
5,83,67.0,27.560000,0,Caucasian,M,190.5,0.000694,100.0,0,...,3.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
10,77,72.0,28.257052,1,Hispanic,F,154.9,0.004861,67.8,1,...,4.2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
17,118,46.0,25.845717,0,Hispanic,M,167.6,0.000000,72.6,0,...,4.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91698,195,47.0,51.439842,1,Caucasian,M,195.0,0.033333,186.0,1,...,4.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91706,121,54.0,19.770448,0,Native American,M,177.8,0.025694,62.5,0,...,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91708,30,75.0,23.060250,0,Caucasian,M,177.8,0.298611,72.9,0,...,4.2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
91709,121,56.0,47.179671,0,Caucasian,F,183.0,0.120139,158.0,0,...,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [24]:
df.dropna().info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62498 entries, 0 to 91712
Data columns (total 72 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   hospital_id                  62498 non-null  int64  
 1   age                          62498 non-null  float64
 2   bmi                          62498 non-null  float64
 3   elective_surgery             62498 non-null  int64  
 4   ethnicity                    62498 non-null  object 
 5   gender                       62498 non-null  object 
 6   height                       62498 non-null  float64
 7   pre_icu_los_days             62498 non-null  float64
 8   weight                       62498 non-null  float64
 9   apache_post_operative        62498 non-null  int64  
 10  arf_apache                   62498 non-null  float64
 11  gcs_eyes_apache              62498 non-null  float64
 12  gcs_motor_apache             62498 non-null  float64
 13  gcs_unable_apach

<ipython-input-24-ab0e0c677e6f>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.dropna().info(verbose = True, null_counts = True)
